In [25]:
# Import necessary libraries for data handling, machine learning and visualization
import numpy as np
import tensorflow as tf
from tensorflow.keras import callbacks
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Input, Dense, Flatten
import matplotlib.image as img

In [26]:
# Create a training dataset from a directory of images, resizing them to 224x224, with 20% of the data used for validation
train_ds = tf.keras.utils.image_dataset_from_directory(
    directory ='C:/Users/User/Allergy-Pal/kaggle/input/food41/images',
    image_size = (224, 224),
    validation_split = 0.2, # Splitting 20% of data for validation
    subset = "training", # Specify this as the training subset
    seed = 42,
    color_mode = 'rgb', # Images are in RGB format
    shuffle = True , # Shuffle images during loading
    label_mode = "categorical" , # Labels are categorical
    batch_size = 32 # Process images in batches of 32
)

# Create a validation dataset using the same directory and parameters as the training dataset
val_ds = tf.keras.utils.image_dataset_from_directory(
    directory = 'C:/Users/User/Allergy-Pal/kaggle/input/food41/images',
    image_size = (224, 224),
    validation_split = 0.2,
    subset = "validation", # Specify this as the validation subset
    seed = 42,
    color_mode = 'rgb',
    label_mode = "categorical" , 
    batch_size = 32
)

Found 101000 files belonging to 101 classes.
Using 80800 files for training.
Found 101000 files belonging to 101 classes.
Using 20200 files for validation.


In [27]:
# Load the class names from a text file
classes = []
with open("C:/Users/User/Allergy-Pal/kaggle/input/food41/meta/meta/classes.txt") as f:
    for line in f.readlines():
        classes.append(line.split("\n")[0]) # Strip out newline characters from the class names

print(classes)

['apple_pie', 'baby_back_ribs', 'baklava', 'beef_carpaccio', 'beef_tartare', 'beet_salad', 'beignets', 'bibimbap', 'bread_pudding', 'breakfast_burrito', 'bruschetta', 'caesar_salad', 'cannoli', 'caprese_salad', 'carrot_cake', 'ceviche', 'cheesecake', 'cheese_plate', 'chicken_curry', 'chicken_quesadilla', 'chicken_wings', 'chocolate_cake', 'chocolate_mousse', 'churros', 'clam_chowder', 'club_sandwich', 'crab_cakes', 'creme_brulee', 'croque_madame', 'cup_cakes', 'deviled_eggs', 'donuts', 'dumplings', 'edamame', 'eggs_benedict', 'escargots', 'falafel', 'filet_mignon', 'fish_and_chips', 'foie_gras', 'french_fries', 'french_onion_soup', 'french_toast', 'fried_calamari', 'fried_rice', 'frozen_yogurt', 'garlic_bread', 'gnocchi', 'greek_salad', 'grilled_cheese_sandwich', 'grilled_salmon', 'guacamole', 'gyoza', 'hamburger', 'hot_and_sour_soup', 'hot_dog', 'huevos_rancheros', 'hummus', 'ice_cream', 'lasagna', 'lobster_bisque', 'lobster_roll_sandwich', 'macaroni_and_cheese', 'macarons', 'miso_sou

In [10]:
img_size = (180 , 180) # Set image size for future use
batch_size = 32 # Define batch size

# Define a list of callback functions to improve model performance
callback_list=[
    # EarlyStopping stops training when validation accuracy doesn't improve for 10 epochs
    callbacks.EarlyStopping(monitor="val_accuracy",patience=10,restore_best_weights=True),
    # Reduce learning rate when validation accuracy plateaus for 3 epochs
    callbacks.ReduceLROnPlateau(factor=0.8,monitor="val_accuracy",patience=3)
]

In [11]:
# Load the base model without its final layers, using input images of size 224x224
base_model = tf.keras.applications.EfficientNetB0(include_top = False , input_shape=(224 , 224 , 3))  

# Freeze the base model's layers so they are not updated during training
base_model.trainable = False 

# Build a custom model using the base model
inputs = tf.keras.layers.Input(shape = (224 , 224 , 3) , name = "Input_layer") # Define the input layer
x = base_model(inputs) # Pass input through the base model
x = tf.keras.layers.Dense(101)(x) # Add a Dense layer with 101 units (for 101 classes)
x = tf.keras.layers.Flatten()(x) # Flatten the output
outputs = tf.keras.layers.Dense(101, activation = "softmax", name = "Output_layer")(x) # Output layer with softmax activatio

# Create the final model
Model_1 = tf.keras.Model(inputs, outputs) 

# Compile the model with categorical cross-entropy loss and Adam optimizer
Model_1.compile(loss = tf.keras.losses.categorical_crossentropy, optimizer = tf.keras.optimizers.Adam(), metrics = ['accuracy'] )

# Train the model for 6 epochs
Model_1_History = Model_1.fit(train_ds, validation_data = val_ds, epochs = 6,verbose = 1, callbacks = callback_list)

Epoch 1/6
2525/2525 ━━━━━━━━━━━━━━━━━━━━ 1368s 537ms/step - accuracy: 0.3579 - loss: 3.4313 - val_accuracy: 0.4625 - val_loss: 3.5084 - learning_rate: 0.0010
Epoch 2/6
2525/2525 ━━━━━━━━━━━━━━━━━━━━ 3670s 1s/step - accuracy: 0.5338 - loss: 2.7924 - val_accuracy: 0.4614 - val_loss: 4.8179 - learning_rate: 0.0010
Epoch 3/6
2525/2525 ━━━━━━━━━━━━━━━━━━━━ 1740s 689ms/step - accuracy: 0.5959 - loss: 2.8101 - val_accuracy: 0.4721 - val_loss: 6.1576 - learning_rate: 0.0010
Epoch 4/6
2525/2525 ━━━━━━━━━━━━━━━━━━━━ 1731s 686ms/step - accuracy: 0.6475 - loss: 2.7493 - val_accuracy: 0.4765 - val_loss: 7.2862 - learning_rate: 0.0010
Epoch 5/6
2525/2525 ━━━━━━━━━━━━━━━━━━━━ 1748s 692ms/step - accuracy: 0.6931 - loss: 2.6498 - val_accuracy: 0.4734 - val_loss: 9.1260 - learning_rate: 0.0010
Epoch 6/6
2525/2525 ━━━━━━━━━━━━━━━━━━━━ 1739s 689ms/step - accuracy: 0.7188 - loss: 2.6924 - val_accuracy: 0.4859 - val_loss: 10.4104 - learning_rate: 0.0010


In [12]:
# Unfreeze the base model for fine-tuning (except the last 10 layers)
base_model.trainable = True
for layer in base_model.layers[:-10]:
    layer.trainable = False

# Compile the model with a lower learning rate for fine-tuning
Model_1.compile(loss = "categorical_crossentropy", optimizer = tf.keras.optimizers.Adam(learning_rate= 0.0001), metrics = ["accuracy"])

# Set a total of 5 more epochs for fine-tuning
initial_epoch = 5
Fine_Tune_epoch = initial_epoch + 5

# Continue training and fine-tuning the model
Stage_2_history = Model_1.fit(train_ds, epochs = Fine_Tune_epoch, validation_data = val_ds, validation_steps = len(val_ds), initial_epoch = initial_epoch-1)

Epoch 5/10
2525/2525 ━━━━━━━━━━━━━━━━━━━━ 1855s 728ms/step - accuracy: 0.7412 - loss: 2.5725 - val_accuracy: 0.5276 - val_loss: 8.6143
Epoch 6/10
2525/2525 ━━━━━━━━━━━━━━━━━━━━ 1496s 592ms/step - accuracy: 0.8465 - loss: 1.0789
Epoch 7/10


C:\Users\User\anaconda3\lib\contextlib.py:137: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


2525/2525 ━━━━━━━━━━━━━━━━━━━━ 1835s 727ms/step - accuracy: 0.8845 - loss: 0.7045 - val_accuracy: 0.5386 - val_loss: 7.8573
Epoch 8/10
2525/2525 ━━━━━━━━━━━━━━━━━━━━ 1631s 646ms/step - accuracy: 0.9044 - loss: 0.5240
Epoch 9/10
2525/2525 ━━━━━━━━━━━━━━━━━━━━ 1851s 733ms/step - accuracy: 0.9239 - loss: 0.3979 - val_accuracy: 0.5472 - val_loss: 7.6334
Epoch 10/10
2525/2525 ━━━━━━━━━━━━━━━━━━━━ 1506s 596ms/step - accuracy: 0.9330 - loss: 0.3341


In [24]:
# Load an image from file and preprocess it for prediction
img = tf.keras.utils.load_img("C:/Users/User/Downloads/steak123.jpg", target_size=(224, 224)) # Resize image to 224x224
img_array = tf.keras.utils.img_to_array(img) # Convert image to array
img_array = tf.expand_dims(img_array, 0) # Add a batch dimension

# Make predictions on the preprocessed image
predictions = Model_1.predict(img_array)
score = tf.nn.softmax(predictions[0]) # Apply softmax to get probabilities

# Print the class prediction and confidence score
print("This image most likely belongs to {} with a {:.2f}% percent confidence.".format(classes[np.argmax(score)], 100 * np.max(predictions[0])))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 294ms/step
This image most likely belongs to steak with a 96.83% percent confidence.
